In [ ]:
import sys
from pathlib import Path

AVES_ROOT = Path("../..") if not "google.colab" in sys.modules else Path("aves_git")

EOD_PATH = AVES_ROOT / "data" / "external" / "EOD_STGO"
EOD_PATH

In [ ]:
OSM_PATH = AVES_ROOT / "data" / "external" / "OSM"

osm_clipped_file = OSM_PATH / 'clipped-scl-osm.pbf'

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd

# esto configura la calidad de la imagen. dependerá de tu resolución. el valor por omisión es 80
mpl.rcParams["figure.dpi"] = 150
# esto depende de las fuentes que tengas instaladas en el sistema.
mpl.rcParams["font.family"] = "Fira Sans Extra Condensed"

In [ ]:
from aves.data import eod

zones = gpd.read_file(AVES_ROOT / "data" / "processed" / "scl_zonas_urbanas.json")

viajes = (
    eod.read_trips(EOD_PATH)
    .merge(eod.read_people(EOD_PATH))
    .merge(eod.read_homes(EOD_PATH))
)

viajes["PesoLaboral"] = viajes["FactorLaboralNormal"] * viajes["Factor_LaboralNormal"]

viajes = viajes[pd.notnull(viajes["PesoLaboral"])]

viajes.columns

In [ ]:
import pyrosm

osm = pyrosm.OSM(str(osm_clipped_file))

In [ ]:
od_zonas = (
    viajes[viajes["Proposito"] == 'Al trabajo']
    .groupby(["Periodo", "ZonaOrigen", "ZonaDestino"])["PesoLaboral"]
    .sum()
    .rename("n_viajes")
    .to_frame()
)
od_zonas.sample(10)


In [ ]:
pois = osm.get_pois()

In [ ]:
from aves.features.osm.pois import categorize_pois

cat_pois = categorize_pois(pois)
cat_pois_area = gpd.sjoin(cat_pois, zones[['NOM_COMUNA', 'geometry']], op='within')
cat_pois_area

In [ ]:
zone_pois = cat_pois_area.groupby('index_right')['taxonomy'].value_counts().unstack(fill_value=0)
zone_pois.sample(10)

In [ ]:
from aves.data.eod import read_homes


hogares = read_homes(EOD_PATH)
hogares.head()

In [ ]:
from aves.features.utils import weighted_mean

zona_ingreso = hogares.groupby('Zona').apply(lambda x: weighted_mean(x, 'IngresoHogar', 'FactorHogar')).rename('ingreso')

In [ ]:
from aves.data.eod import read_people


personas = read_people(EOD_PATH).merge(hogares)
zona_poblacion =personas.groupby('Zona')['FactorPersona'].sum().rename('population')
zona_poblacion

In [ ]:
zones.join(np.log(zona_ingreso + 1), on='ID').plot(column='ingreso')

In [ ]:
zones.join(np.log(zona_poblacion + 1), on='ID').plot(column='population')

In [ ]:
zone_pois_filtered = zone_pois[[c for c in zone_pois.columns if not ':' in c]].sum(axis=1).rename('total_poi')
zone_pois_filtered

In [ ]:
zone_features = zona_ingreso.to_frame().join(zona_poblacion).join(zone_pois_filtered).fillna(0)
zone_features

In [ ]:
from cytoolz import valmap

dst_pos = valmap(lambda x: (x.x, x.y), zones.to_crs('epsg:5361').set_index('ID').centroid.to_dict())
dst_pos

In [ ]:
from scipy.spatial.distance import cdist

distance_matrix = cdist(np.array(list(dst_pos.values())), np.array(list(dst_pos.values())))
distance_matrix.shape

In [ ]:
distance_df = pd.DataFrame(distance_matrix / 1000, index=list(dst_pos.keys()), columns=list(dst_pos.keys())).stack().reset_index()
distance_df.columns = ['ZonaOrigen', 'ZonaDestino', 'distance']
distance_df.head()

In [ ]:
model_features = (
    od_zonas.loc["Punta Mañana 2 (7:31 - 9:00)"]
    .reset_index()
    .join(zone_features.add_prefix("o_"), on="ZonaOrigen")
    .join(zone_features.add_prefix("d_"), on="ZonaDestino")
    .fillna(0)
    .merge(distance_df)
)

model_features


In [ ]:
model_features.columns

In [ ]:
model_features.describe()

In [ ]:
model_features_clean = model_features[model_features["n_viajes"] > 10]

In [ ]:
import statsmodels.formula.api as sm

sm.poisson(
    "np.log(n_viajes + 1) ~ distance + np.sqrt(o_population) + np.sqrt(d_population) + np.log(o_ingreso + 1) + np.log(d_ingreso + 1) + np.sqrt(o_total_poi) + np.sqrt(d_total_poi)",
    data=model_features_clean,
).fit().summary()


In [ ]:
import spglm
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from patsy import dmatrices, NAAction

In [ ]:
model_features_clean['ZonaOrigen']

In [ ]:
len(model_features_clean['ZonaOrigen'].values)

In [ ]:
zones.to_crs('epsg:5361').set_index('ID').centroid.sort_index().loc[[1,1,1]]

In [ ]:
coords = zones.to_crs('epsg:5361').dissolve('ID').centroid.loc[model_features_clean['ZonaOrigen'].values]
coords = np.vstack([coords.x.values, coords.y.values]).T
len(coords), len(model_features_clean)

In [ ]:
y, X = dmatrices(
    "n_viajes ~ distance + np.sqrt(o_population) + np.sqrt(d_population) + np.log(o_ingreso + 1) + np.log(d_ingreso + 1) + np.sqrt(o_total_poi) + np.sqrt(d_total_poi)",
    model_features_clean,
    NA_action="raise",
)

len(y), len(X)


In [ ]:
sample_idx = np.random.choice(np.arange(len(model_features_clean)), size=500, replace=False)
#sample_idx

In [ ]:
from spglm.family import Poisson

In [ ]:
import multiprocessing

gwr_selector = Sel_BW(coords[sample_idx], np.asarray(y)[sample_idx], np.asarray(X)[sample_idx], family=Poisson(), multi=False, fixed=False, kernel='bisquare')
gwr_bw = gwr_selector.search(bw_min=500, pool=multiprocessing.Pool(8))
print(gwr_bw)

In [ ]:
gwr_model = GWR(
    coords,
    np.asarray(y),
    np.asarray(X),
    gwr_bw,
    fixed=False,
    kernel="bisquare",
    family=Poisson(),
)
gwr_results = gwr_model.fit(pool=multiprocessing.Pool(8))


In [ ]:
gwr_results.summary()

In [ ]:
gwr_results.params.shape

In [ ]:
X.design_info.term_names

In [ ]:
gwr_df = pd.DataFrame(gwr_results.params, index=model_features_clean['ZonaOrigen'].values, columns=X.design_info.term_names).drop_duplicates()
gwr_df

In [ ]:
gwr_df[['distance', 'np.log(d_ingreso + 1)']].plot(kind='kde')

In [ ]:
from aves.visualization.figures import figure_from_geodataframe
from aves.visualization.maps import choropleth_map

fig, ax = figure_from_geodataframe(zones, height=7)

factor = 'np.sqrt(d_total_poi)'

choropleth_map(ax, zones.join(gwr_df), factor, linewidth=0, cbar_args=dict(
        label="Factor",
        height="22%",
        width="2%",
        orientation="vertical",
        location="center left",
        label_size="small",
        bbox_to_anchor=(0.0, 0.0, 0.9, 1.0),
    ))